# 1. GCI 데이터 변환

In [1]:
import pandas as pd
from pandas import Series, DataFrame
import os
from datetime import datetime, timedelta
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
#기본 경로 선언
source_path = '../data1/'
output_path = '../data2/'

gci_filenm = 'raw_gci.csv'
pds_filenm = 'raw_index_pds.csv'
var_filenm = '로이터_변수_데이터set_180605.xlsx'
idx_filenm = '로이터_지표_인도_오스트리아_인디아_아프리카_180614_원본.xlsx'

gci_path = os.path.join(source_path, gci_filenm)
pds_path = os.path.join(source_path, pds_filenm)
var_path = os.path.join(source_path, var_filenm)
idx_path = os.path.join(source_path, idx_filenm)

gci_output_path = os.path.join(output_path, 'weekly_gci.csv')
pds_output_path = os.path.join(output_path, 'weekly_pds.csv')
var_output_path = os.path.join(output_path, 'var_reuter_set.csv')
idx_output_path = os.path.join(output_path, 'idx_reuter_coal.csv')

In [3]:
# csv 파일 불러오기
df_gci = pd.read_csv(gci_path, parse_dates=True)
# 컬럼명 변경, 데이터(Date)형식 변경 및 Index 컬럼 변경
df_gci.columns = ['Date', 'GCI']
df_gci['Date'] = pd.to_datetime(df_gci['Date'])
df_gci = df_gci.set_index('Date')

#현재 변수에 있는 내용 보여주기
df_gci.head()

,GCI
Date,
2003-07-04,24.63
2003-07-11,24.59
2003-07-18,24.82
2003-07-25,24.89
2003-08-01,24.91


In [4]:
#변경된 csv 저장
df_gci.to_csv(gci_output_path)

# 2. KoreaPDS 데이터 변환

In [5]:
# csv 파일 불러오기
df_pds = pd.read_csv(pds_path, index_col=0, parse_dates=True)

#pds csv의 row와 column수 출력
df_pds.shape

(602, 107)

In [6]:
#Bloomberg, KoreaPDF 지수 제거
df_pds = df_pds.drop(columns=df_pds.filter(regex='PDS|Bloomberg'))
df_pds.shape

(602, 98)

In [7]:
df_pds

,WorldBank-Agriculture Index,IMF-Crude Oil(petroleum) Price index,CRB BLS Spot Index,철근 [SHFE] 2018.10,LMEX Index,GSCI Light Energy Index (CPW 4),GSCI Livestock,WorldBank-Energy Index,중후판 12(mm)x8(ft)x20(ft) [국내(1차 유통가)] 현물,원당 raw sugar [ICE(NYBOT)] 2018.10,...,Baltic Dirty Tanker Index (BDTI),GSCI Precious Metals,백금 [LPPM] 현물,팔라듐 [LPPM] 현물,IMF-Metals Price Index,IMF-Fuel (energy) Index,Howe Robinson Container Index,GSCI Petroleum,CRB BLS Metals Sub-Index,CRB BLS Fat & Oils Sub-Index
Date,,,,,,,,,,,,,,,,,,,,,
2007-01-05,73.6850,102.4100,359.17,3577.00,3459.10,550.84,267.27,74.7550,700000.0,11.07,...,1217.800,1988.83,1132.80,330.10,171.5875,104.0725,1027.0,329.48,686.3950,273.0175
2007-01-12,74.1500,104.3000,363.91,3577.00,3439.80,550.84,267.27,76.3000,700000.0,10.81,...,1208.200,1988.83,1147.20,334.40,172.3950,105.7850,1035.0,329.48,695.0500,275.6350
2007-01-19,74.6150,106.1900,367.04,3577.00,3579.16,550.84,267.27,77.8450,700000.0,10.78,...,1241.600,1988.83,1168.60,347.10,173.2025,107.4975,1060.4,329.48,703.7050,278.2525
2007-01-26,75.0800,108.0800,371.99,3577.00,3465.72,550.84,267.27,79.3900,700000.0,10.57,...,1203.800,1988.83,1171.80,337.20,174.0100,109.2100,1086.7,329.48,712.3600,280.8700
2007-02-02,75.0525,109.5225,365.30,3577.00,3378.16,550.84,267.27,79.9300,700000.0,10.31,...,1174.000,1988.83,1185.80,339.20,176.4475,110.4300,1093.4,329.48,727.5525,291.6200
2007-02-09,75.0250,110.9650,371.03,3577.00,3523.44,550.84,267.27,80.4700,700000.0,10.71,...,1187.800,1988.83,1201.40,339.20,178.8850,111.6500,1118.9,329.48,742.7450,302.3700
2007-02-16,74.9975,112.4075,374.32,3577.00,3659.80,550.84,267.27,81.0100,700000.0,10.63,...,1164.200,1988.83,1218.80,342.60,181.3225,112.8700,1132.4,329.48,757.9375,313.1200
2007-02-23,74.9700,113.8500,386.91,3577.00,3737.76,550.84,267.27,81.5500,700000.0,10.85,...,1106.600,1988.83,1231.20,350.05,183.7600,114.0900,1140.4,329.48,773.1300,323.8700
2007-03-02,75.1375,115.9575,379.99,3577.00,3657.62,550.84,267.27,82.8600,700000.0,10.64,...,1068.400,1988.83,1187.80,346.00,188.3075,115.8425,1141.6,329.48,779.7475,322.3825


In [8]:
#인코딩 후 저장
df_pds.to_csv(pds_output_path, encoding='ms949')

# 3. Reuter 데이터 변환

In [9]:
df_pre_reuter = pd.read_excel(var_path)
df_reuter1 = df_pre_reuter.iloc[1:, 24:26]
df_reuter1.columns = ['Date', 'OILFUTURES=']
df_reuter2 = df_pre_reuter.iloc[1:, 0:2]
df_reuter2.columns = ['Date', 'USFFTARGET= CHT']
df_reuter3 = df_pre_reuter.iloc[1:, 3:5]
df_reuter3.columns = ['Date', 'USD=']
df_reuter4 = df_pre_reuter.iloc[1:, 6:8]
df_reuter4.columns = ['Date', 'XAU=']
df_reuter5 = df_pre_reuter.iloc[1:, 9:11]
df_reuter5.columns = ['Date', 'CNPMIC=ECI']
df_reuter6 = df_pre_reuter.iloc[1:, 12:14]
df_reuter6.columns = ['Date', 'JPRPMI=ECI']
df_reuter7 = df_pre_reuter.iloc[1:, 15:17]
df_reuter7.columns = ['Date', 'KRBUSI=ECI']
df_reuter8 = df_pre_reuter.iloc[1:, 18:20]
df_reuter8.columns = ['Date', 'USPMI=ECI']
df_reuter9 = df_pre_reuter.iloc[1:, 21:23]
df_reuter9.columns = ['Date', 'WTI=']
df_reuter10 = df_pre_reuter.iloc[1:, 27:29]
df_reuter10.columns = ['Date', 'GCI=']
df_reuter11 = df_pre_reuter.iloc[1:, 30:32]
df_reuter11.columns = ['Date', 'CCE=STOCK']
df_reuter = pd.merge(df_reuter1, df_reuter2, how='left')
df_reuter = pd.merge(df_reuter, df_reuter3, how='left')
df_reuter = pd.merge(df_reuter, df_reuter4, how='left')
df_reuter = pd.merge(df_reuter, df_reuter5, how='left')
df_reuter = pd.merge(df_reuter, df_reuter6, how='left')
df_reuter = pd.merge(df_reuter, df_reuter7, how='left')
df_reuter = pd.merge(df_reuter, df_reuter8, how='left')
df_reuter = pd.merge(df_reuter, df_reuter9, how='left')
df_reuter = pd.merge(df_reuter, df_reuter10, how='left')
df_reuter = pd.merge(df_reuter, df_reuter11, how='left')
df_reuter = df_reuter.drop_duplicates(['Date'], keep='last')
df_reuter = df_reuter.set_index('Date')
df_reuter = df_reuter.sort_values(by=['Date'])
df_reuter

,OILFUTURES=,USFFTARGET= CHT,USD=,XAU=,CNPMIC=ECI,JPRPMI=ECI,KRBUSI=ECI,USPMI=ECI,WTI=,GCI=,CCE=STOCK
Date,,,,,,,,,,,
1998-06-30,14.18,5.5,1370,296.7,NaN,NaN,NaN,48.9,14.3,NaN,NaN
1998-07-31,14.21,5.5,1229,286.2,NaN,NaN,NaN,49.2,14.27,NaN,NaN
1998-08-31,13.34,5.5,1348,276.4,NaN,NaN,NaN,49.3,13.29,NaN,NaN
1998-09-30,16.14,5.25,1388.5,296.7,NaN,NaN,NaN,48.7,16.19,NaN,NaN
1998-10-31,14.42,5,1318,292.3,NaN,NaN,NaN,48.7,14.48,NaN,NaN
1998-11-30,11.22,4.75,1245,292.7,NaN,NaN,NaN,48.2,11.37,NaN,NaN
1998-12-31,12.05,4.75,1200,288,NaN,NaN,NaN,46.8,12.14,NaN,NaN
1999-01-31,12.75,4.75,1174,286.1,NaN,NaN,NaN,50.6,12.81,NaN,NaN
1999-02-28,12.27,4.75,1222.5,286.6,NaN,NaN,NaN,51.7,12.31,NaN,NaN


In [10]:

# df_reuter.shaped
# df_reuter.interpolate(method='linear', limit_direction='both', axis=0)
for c in df_reuter:
    df_reuter[c] = df_reuter[c].astype('float')
    df_reuter[c] = df_reuter[c].interpolate(limit_direction='both')
#     df_reuter[c].dtypes
df_reuter
# for c in df_reuter:
#    df_reuter[c] = df_reuter[c].interpolate(limit_direction='both')

,OILFUTURES=,USFFTARGET= CHT,USD=,XAU=,CNPMIC=ECI,JPRPMI=ECI,KRBUSI=ECI,USPMI=ECI,WTI=,GCI=,CCE=STOCK
Date,,,,,,,,,,,
1998-06-30,14.18,5.50,1370.000000,296.700000,49.400000,51.700000,87.000000,48.900000,14.30,106.283359,5.050000
1998-07-31,14.21,5.50,1229.000000,286.200000,49.400000,51.700000,87.000000,49.200000,14.27,106.283359,5.050000
1998-08-31,13.34,5.50,1348.000000,276.400000,49.400000,51.700000,87.000000,49.300000,13.29,106.283359,5.050000
1998-09-30,16.14,5.25,1388.500000,296.700000,49.400000,51.700000,87.000000,48.700000,16.19,106.283359,5.050000
1998-10-31,14.42,5.00,1318.000000,292.300000,49.400000,51.700000,87.000000,48.700000,14.48,106.283359,5.050000
1998-11-30,11.22,4.75,1245.000000,292.700000,49.400000,51.700000,87.000000,48.200000,11.37,106.283359,5.050000
1998-12-31,12.05,4.75,1200.000000,288.000000,49.400000,51.700000,87.000000,46.800000,12.14,106.283359,5.050000
1999-01-31,12.75,4.75,1174.000000,286.100000,49.400000,51.700000,87.000000,50.600000,12.81,106.283359,5.050000
1999-02-28,12.27,4.75,1222.500000,286.600000,49.400000,51.700000,87.000000,51.700000,12.31,106.283359,5.050000


In [11]:
df_reuter

,OILFUTURES=,USFFTARGET= CHT,USD=,XAU=,CNPMIC=ECI,JPRPMI=ECI,KRBUSI=ECI,USPMI=ECI,WTI=,GCI=,CCE=STOCK
Date,,,,,,,,,,,
1998-06-30,14.18,5.50,1370.000000,296.700000,49.400000,51.700000,87.000000,48.900000,14.30,106.283359,5.050000
1998-07-31,14.21,5.50,1229.000000,286.200000,49.400000,51.700000,87.000000,49.200000,14.27,106.283359,5.050000
1998-08-31,13.34,5.50,1348.000000,276.400000,49.400000,51.700000,87.000000,49.300000,13.29,106.283359,5.050000
1998-09-30,16.14,5.25,1388.500000,296.700000,49.400000,51.700000,87.000000,48.700000,16.19,106.283359,5.050000
1998-10-31,14.42,5.00,1318.000000,292.300000,49.400000,51.700000,87.000000,48.700000,14.48,106.283359,5.050000
1998-11-30,11.22,4.75,1245.000000,292.700000,49.400000,51.700000,87.000000,48.200000,11.37,106.283359,5.050000
1998-12-31,12.05,4.75,1200.000000,288.000000,49.400000,51.700000,87.000000,46.800000,12.14,106.283359,5.050000
1999-01-31,12.75,4.75,1174.000000,286.100000,49.400000,51.700000,87.000000,50.600000,12.81,106.283359,5.050000
1999-02-28,12.27,4.75,1222.500000,286.600000,49.400000,51.700000,87.000000,51.700000,12.31,106.283359,5.050000


In [12]:
#df_reuter = df_reuter.loc['2015-6':'2018-6']
df_reuter = df_reuter.resample('W').bfill()
df_reuter

,OILFUTURES=,USFFTARGET= CHT,USD=,XAU=,CNPMIC=ECI,JPRPMI=ECI,KRBUSI=ECI,USPMI=ECI,WTI=,GCI=,CCE=STOCK
Date,,,,,,,,,,,
1998-07-05,14.21,5.500000,1229.000000,286.200000,49.400000,51.700000,87.000000,49.200000,14.27,106.283359,5.050000
1998-07-12,14.21,5.500000,1229.000000,286.200000,49.400000,51.700000,87.000000,49.200000,14.27,106.283359,5.050000
1998-07-19,14.21,5.500000,1229.000000,286.200000,49.400000,51.700000,87.000000,49.200000,14.27,106.283359,5.050000
1998-07-26,14.21,5.500000,1229.000000,286.200000,49.400000,51.700000,87.000000,49.200000,14.27,106.283359,5.050000
1998-08-02,13.34,5.500000,1348.000000,276.400000,49.400000,51.700000,87.000000,49.300000,13.29,106.283359,5.050000
1998-08-09,13.34,5.500000,1348.000000,276.400000,49.400000,51.700000,87.000000,49.300000,13.29,106.283359,5.050000
1998-08-16,13.34,5.500000,1348.000000,276.400000,49.400000,51.700000,87.000000,49.300000,13.29,106.283359,5.050000
1998-08-23,13.34,5.500000,1348.000000,276.400000,49.400000,51.700000,87.000000,49.300000,13.29,106.283359,5.050000
1998-08-30,13.34,5.500000,1348.000000,276.400000,49.400000,51.700000,87.000000,49.300000,13.29,106.283359,5.050000


In [13]:
#인코딩 후 저장
df_reuter.to_csv(var_output_path, encoding='utf-8')

# 4. Reuter 석탄 가격 지표

In [14]:
df_pre_reuter_coal = pd.read_excel(idx_path)
df_reu_coal1 = df_pre_reuter_coal.iloc[1:, 0:2]
df_reu_coal1.columns = ['Date', 'China Export Coal Current Price']
df_reu_coal2 = df_pre_reuter_coal.iloc[1:, 3:5]
df_reu_coal2.columns = ['Date', 'China Export Other Coal Volume']
df_reu_coal3 = df_pre_reuter_coal.iloc[1:, 6:8]
df_reu_coal3.columns = ['Date', 'CHina Import Coal Volume']
df_reu_coal4 = df_pre_reuter_coal.iloc[1:, 9:11]
df_reu_coal4.columns = ['Date', 'China Import Coal Coke&Briquettes Current Price']
df_reu_coal5 = df_pre_reuter_coal.iloc[1:, 12:14]
df_reu_coal5.columns = ['Date', 'India Import Coal Coke&Briquettes Current Price']
df_reu_coal6 = df_pre_reuter_coal.iloc[1:, 15:17]
df_reu_coal6.columns = ['Date', 'SouthAfrica Producer Price Mining Coal&Gas Price']
df_reu_coal = pd.merge(df_reu_coal1, df_reu_coal2, how='left')
df_reu_coal = pd.merge(df_reu_coal, df_reu_coal3, how='left')
df_reu_coal = pd.merge(df_reu_coal, df_reu_coal4, how='left')
df_reu_coal = pd.merge(df_reu_coal, df_reu_coal5, how='left')
df_reu_coal = pd.merge(df_reu_coal, df_reu_coal6, how='left')
df_reu_coal = df_reu_coal.set_index('Date')
df_reu_coal

,China Export Coal Current Price,China Export Other Coal Volume,CHina Import Coal Volume,China Import Coal Coke&Briquettes Current Price,India Import Coal Coke&Briquettes Current Price,SouthAfrica Producer Price Mining Coal&Gas Price
Date,,,,,,
1998-06-30,62138000,1980000,80000,3191000,NaN,NaN
1998-07-31,66274000,2040000,140000,5564000,NaN,NaN
1998-08-31,90068000,2690000,280000,12534000,NaN,NaN
1998-09-30,82114000,2510000,170000,7609000,NaN,NaN
1998-10-31,83505000,2590000,210000,8872000,NaN,NaN
1998-11-30,89736000,2770000,130000,5598000,NaN,NaN
1998-12-31,204026000,6470000,190000,8179000,NaN,NaN
1999-01-31,50663000,1640000,230000,9497000,NaN,NaN
1999-02-28,82197000,2700000,40000,1582000,NaN,NaN


In [15]:
for c in df_reu_coal:
    df_reu_coal[c] = df_reu_coal[c].astype('float')
    df_reu_coal[c] = df_reu_coal[c].interpolate(limit_direction='both')

In [16]:
#인코딩 후 저장
df_reu_coal.to_csv(idx_output_path, encoding='utf-8')

# 5. 정규화

In [17]:
def normalize_(df):
    values = df.values
    values = np.asarray(values, dtype=np.float32)

    scaler = MinMaxScaler(feature_range=(0, 1))
    values = scaler.fit_transform(values)
    df = pd.DataFrame(values, index=df.index, columns=df.columns)

    return df, scaler

In [18]:
# df_reu_coal_scaler.transform(df_reu_coal)
# df_reu_coal_scaler.data_min_
# df_reu_coal_scaler.data_max_
df_gci_normalize, df_gci_scaler = normalize_(df_gci)
df_pds_normalize, df_pds_scaler = normalize_(df_pds)
df_reuter_normalize, df_reuter_scaler = normalize_(df_reuter)
df_reu_coal_normalize, df_reu_coal_scaler = normalize_(df_reu_coal)

In [19]:
# np.savetxt('../data2/scaler_test.csv', df_reu_coal_scaler.inverse_transform(df_reu_coal_normalize),  delimiter=',')
np.savetxt('../data2/weekly_gci_normalize.csv', df_gci_normalize,  delimiter=',')
np.savetxt('../data2/weekly_pds_normalize.csv', df_pds_normalize,  delimiter=',')
np.savetxt('../data2/var_reuter_set_normalize.csv', df_reuter_normalize,  delimiter=',')
np.savetxt('../data2/idx_reuter_coal_normalize.csv', df_reu_coal_normalize,  delimiter=',')